In [ ]:
cd "/content/drive/MyDrive/University/Year 4/Semester B/Foundations-of-Deep-Learning/foundations-dl-hw1/"

/content/drive/MyDrive/University/Year 4/Semester B/Foundations-of-Deep-Learning/foundations-dl-hw1


In [ ]:
import torch
import itertools
import pandas as pd

from src.train import train
from src.cifar10_dataset import trainloader, testloader
from src.models import BaselineNN
from src.utils import init_func__zero_mean_gaussian, get_sgd_optimizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

>> Initializing data with parameters: BATCH_SIZE=64, DATA_FRACTION=0.1
Files already downloaded and verified
Files already downloaded and verified


device(type='cuda')

# [0] Run training sanity check

In [ ]:
model = BaselineNN()
results = train(
    model=model,
    init_func=init_func__zero_mean_gaussian(std=1),
    optimizer=get_sgd_optimizer(model, lr=0.001, momentum=0.9),
    trainloader=trainloader,
    testloader=testloader,
    num_epochs=3,
    device=device
)

best_epoch, best_acc = results.test_accuracies.argmax(), results.test_accuracies.max()
print(f"Best test-set accuracy: {best_acc} in epoch {best_epoch}")

results.report()

>> Runs training of BaselineNN(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=256, bias=True)
    (2): Dropout(p=0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
) on device=cuda for 3 epochs.


Evaluate - Epoch 2: 100%|██████████| 16/16 [00:00<00:00, 23.87batch/s, accuracy=0.175, loss=59.4]


Best test-set accuracy: 0.212109375 in epoch 2
>> Accuracies Curves:


>> Losses Curves:


>> Optimization ended with:
   >> TRAIN-SET: accuracy=20.035601265822788%, loss=151.72653845292103
   >> TEST-SET: accuracy=21.2109375%, loss=59.661386489868164


## [1] Grid search for baseline HParams

In [ ]:
# Full grid-search that was saved in CSV
"""
lrs = [0.001, 0.01, 0.1, 0.2, 0.3]
momentums = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
stds = [0.1, 0.5, 1.0]

grid_search_results = []  # params --> best-test-accuracy

for lr, momentum, std in itertools.product(lrs, momentums, stds):
    print(f">> Training with: lr={lr}, momentum={momentum}, std={std} --------------------------")
    model = BaselineNN()
    results = train(
      model=model,
      init_func=init_func__zero_mean_gaussian(std=std),
      optimizer=get_sgd_optimizer(model, lr=lr, momentum=momentum),
      trainloader=trainloader,
      testloader=testloader,
      num_epochs=50,
    )    
    results.get_accuracies_curve().show()

    grid_search_results.append(dict(lr=lr, momentum=momentum, std=std, 
                                    best_test_acc=results.test_accuracies.max(), 
                                    best_train_acc=results.train_accuracies.max()))
# Save results to CSV
pd.DataFrame(grid_search_results).to_csv('./report/q2-baseline-grid-search.csv')
"""

# Grid search results:
pd.read_csv('./report/q2-baseline-grid-search.csv')

,Unnamed: 0,lr,momentum,std,best_test_acc,best_train_acc
0,0,0.001,0.0,0.1,0.178125,0.145767
1,1,0.001,0.0,0.5,0.156836,0.139834
2,2,0.001,0.0,1.0,0.159180,0.141218
3,3,0.001,0.2,0.1,0.211523,0.183742
4,4,0.001,0.2,0.5,0.185547,0.179984
...,...,...,...,...,...,...
85,85,0.300,0.8,0.5,0.112891,0.113133
86,86,0.300,0.8,1.0,0.112891,0.110166
87,87,0.300,0.9,0.1,0.112891,0.110562
88,88,0.300,0.9,0.5,0.112891,0.109771


#### Optimal baseline hyper parameters found:

In [ ]:
def get_optimal_baseline_training_params():
  model = BaselineNN()
  optimizer = get_sgd_optimizer(model, lr=0.001, momentum=0.9)
  baseline_parameters = dict(
    model=model,
    optimizer=optimizer,
    init_func=init_func__zero_mean_gaussian(std=0.1),
    trainloader=trainloader,
    testloader=testloader,
    num_epochs=100,
  )

  return baseline_parameters

results = train(**get_optimal_baseline_training_params())

results.report()

>> Runs training of BaselineNN(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=256, bias=True)
    (2): Dropout(p=0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
) on device=cpu for 100 epochs.


Evaluate - Epoch 99: 100%|██████████| 16/16 [00:00<00:00, 42.21batch/s, accuracy=0.3, loss=1.93]


>> Accuracies Curves:


>> Losses Curves:


>> Optimization ended with:
   >> TRAIN-SET: accuracy=58.26740506329114%, loss=1.2462419875060455
   >> TEST-SET: accuracy=40.05859375%, loss=1.7039632350206375


## [2] Optimization Impact

In [7]:
from src.utils import get_adam_optimizer

# Override the optimizer of the model
for lr in [train_params['optimizer'].defaults['lr'], 0.005]: #[0.0001, 0.001, 0.01, 0.05]
  train_params = get_optimal_baseline_training_params()
  adam_opt = get_adam_optimizer(model=train_params['model'], lr=lr)
  train_params.update({'optimizer': adam_opt})

  print(train_params)
  results = train(**train_params)

  results.report()

{'model': BaselineNN(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=256, bias=True)
    (2): Dropout(p=0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
), 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.005
    maximize: False
    weight_decay: 0
), 'init_func': <function init_func__zero_mean_gaussian.<locals>.func at 0x7ff2faf5a430>, 'trainloader': <torch.utils.data.dataloader.DataLoader object at 0x7ff332dd5430>, 'testloader': <torch.utils.data.dataloader.DataLoader object at 0x7ff330baeb50>, 'num_epochs': 100}
>> Runs training of BaselineNN(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=256, bias=True)
    (2): Dropout(p=0, inplace=Fa

Evaluate - Epoch 99: 100%|██████████| 16/16 [00:00<00:00, 40.99batch/s, accuracy=0.25, loss=3.21]


>> Accuracies Curves:


>> Losses Curves:


>> Optimization ended with:
   >> TRAIN-SET: accuracy=64.16139240506328%, loss=0.9712273787848557
   >> TEST-SET: accuracy=32.71484375%, loss=2.70184288918972
{'model': BaselineNN(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=256, bias=True)
    (2): Dropout(p=0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
), 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.005
    maximize: False
    weight_decay: 0
), 'init_func': <function init_func__zero_mean_gaussian.<locals>.func at 0x7ff2faebc310>, 'trainloader': <torch.utils.data.dataloader.DataLoader object at 0x7ff332dd5430>, 'testloader': <torch.utils.data.dataloader.DataLoader object at 0x7ff330baeb50>, 'num_epochs': 100}
>> Runs training of BaselineNN(
 

Evaluate - Epoch 99: 100%|██████████| 16/16 [00:00<00:00, 38.25batch/s, accuracy=0.3, loss=2.85]


>> Accuracies Curves:


>> Losses Curves:


>> Optimization ended with:
   >> TRAIN-SET: accuracy=60.02768987341772%, loss=1.0974259082275102
   >> TEST-SET: accuracy=32.44140625%, loss=2.730946123600006


## [3] Initialization Impact

In [ ]:
from src.utils import init_func__xavier

# TODO